# Importando bibliotecas e os dados do projeto

In [1]:
# Dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# SKlearn
  # Treino e teste
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

  # Avaliação de performance
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

  # Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Interface
from IPython.display import display
import ipywidgets as widgets

url = "https://media.githubusercontent.com/media/2025-1-NCC5/Projeto3/refs/heads/main/documentos/DataFrame/dadosIntegrados.csv"
df = pd.read_csv(url)

# Separando os dados para passar para a IA

In [2]:
# Retirando a coluna de index duplicada da importação
df.drop('Unnamed: 0', axis=1, inplace=True)

# Verificação inicial do DF
df.head()

,ID_Corrida,ID_Categoria,Dia,Hora,Distancia,Preco
0,1183200,NaN,1,10,21,151.05
1,1183201,NaN,1,10,14,31.50
2,1183202,NaN,1,10,13,42.00
3,1183203,NaN,1,10,19,47.50
4,1183204,NaN,1,10,2,7.50


In [3]:
# Retirando as linhas que não tem ID_Categoria para criar um sample mais preciso
df.dropna(subset=['ID_Categoria'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [4]:
# Transformando o tipo das colunas preço e categoria
df['Preco'] = [int(label) for label in df['Preco']]
df['ID_Categoria'] = [int(label) for label in df['ID_Categoria']]

In [5]:
df.head()

,ID_Corrida,ID_Categoria,Dia,Hora,Distancia,Preco
0,1183334,2,1,10,7,30
1,1183356,2,1,10,7,30
2,1183509,2,1,10,9,32
3,1183851,2,1,13,9,36
4,1186580,2,2,10,7,29


In [6]:
df.count()

,0
ID_Corrida,24714
ID_Categoria,24714
Dia,24714
Hora,24714
Distancia,24714
Preco,24714


# Montando o treino e teste da IA

In [10]:
x = df.drop(['Preco', 'ID_Corrida'], axis=1)
y = df['Preco']

scaler = StandardScaler()
scaled_x = scaler.fit_transform(x)

treino_x, teste_x, treino_y, teste_y = train_test_split(
  scaled_x,y,test_size=0.3, random_state=42)

display(teste_x)
display(teste_y)

array([[-1.11914636,  0.65803385,  1.22873597, -0.08392906],
       [ 0.89087639, -0.71405209,  0.27456461,  0.02632521],
       [-1.11914636, -1.40009505,  1.94436449, -0.09311691],
       ...,
       [-1.11914636,  1.34407682, -1.15669244,  0.04470092],
       [ 0.89087639, -0.71405209,  0.99019313, -0.10230477],
       [ 0.89087639,  0.65803385,  0.51310745, -0.03798978]])

,Preco
4106,16
10834,8
6913,10
1661,46
24071,12
...,...
11037,9
14994,9
5751,53
11176,20


## Fazendo análise de desempenho de diferentes modelo

In [16]:
# Função para fazer a avaliação de um modelo especificado
def avaliacao(modelo, nome):
  modelo.fit(treino_x, treino_y)

  predicao = modelo.predict(teste_x)
  MAE = mean_absolute_error(teste_y, predicao)
  MSE = mean_squared_error(teste_y, predicao)
  RMSE = np.sqrt(MSE)
  R2 = r2_score(teste_y, predicao)

  print(f'Modelo: {nome}\n')
  print(f"MAE: {MAE:.2f}%")
  print(f"MSE: {MSE:.2f}")
  print(f"RMSE: {RMSE:.2f}")
  print(f"R²: {R2:.2f}")
  print("\n------------------------------------------------------------\n")

In [17]:
# Passando diversos modelos pela função de análise de desempenho
avaliacao(LogisticRegression(), "Regressão Logística")
avaliacao(KNeighborsRegressor(n_neighbors=5), "KNN")
avaliacao(DecisionTreeRegressor(), "Árvore de Decisão")
avaliacao(RandomForestRegressor(n_estimators=100), "Random Forest")

Modelo: Regressão Logística

MAE: 20.89%
MSE: 15908.85
RMSE: 126.13
R²: -1.78

------------------------------------------------------------

Modelo: KNN

MAE: 18.08%
MSE: 8271.54
RMSE: 90.95
R²: -0.44

------------------------------------------------------------

Modelo: Árvore de Decisão

MAE: 16.21%
MSE: 7122.78
RMSE: 84.40
R²: -0.24

------------------------------------------------------------

Modelo: Random Forest

MAE: 16.05%
MSE: 6655.12
RMSE: 81.58
R²: -0.16

------------------------------------------------------------



## Definindo o modelo

In [13]:
# Escolhendo o modelo
modelo = RandomForestRegressor(n_estimators=100)

# Treino do modelo escolhido
modelo.fit(treino_x, treino_y)

RandomForestRegressor()

# Construindo a interface para fazer novas previsões

In [26]:
# Função para criar a interface
def interface():
  Categoria = widgets.FloatText(description='Categoria:')
  Dia = widgets.FloatText(description='Dia:')
  Hora = widgets.FloatText(description='Hora:')
  Distancia = widgets.FloatText(description='Distancia:')

  botao = widgets.Button(description='Fazer previsão')

  # Função que define o que acontece quando o botão é clicado
  def click(b):
    entrada = pd.DataFrame([{'ID_Categoria': Categoria.value, 'Dia': Dia.value, 'Hora': Hora.value, 'Distancia': Distancia.value}])

    entrada_scaled = scaler.transform(entrada)
    pred = modelo.predict(entrada_scaled)[0]

    # Verificando qual a categoria de Uber selecionada
    categoria_tipo = ''
    match Categoria.value:
      case 2:
        categoria_tipo = 'UberX'
      case 4:
        categoria_tipo = 'UberBlack'
      case 5:
        categoria_tipo = 'Uber Taxi'
      case 6:
        categoria_tipo = 'Uber Executivo'
      case 8:
        categoria_tipo = 'UberBlack Bag'
      case 9:
        categoria_tipo = 'Uber Comfort'
      case 10:
        categoria_tipo = 'UberFlash'

    print("Resultado\n")
    print(f'Dia: {Dia.value:.0f}')
    print(f'Hora: {Hora.value:.0f}')
    print(f'Categoria: {categoria_tipo}')
    print(f'Dia: {Distancia.value:.0f}')
    print(f'Preço: R${round(pred, 2)}')
    print("\n------------------------------------------------------------\n")

  botao.on_click(click)
  display(Categoria, Dia, Hora, Distancia, botao)

interface()

FloatText(value=0.0, description='Categoria:')

FloatText(value=0.0, description='Dia:')

FloatText(value=0.0, description='Hora:')

FloatText(value=0.0, description='Distancia:')

Button(description='Fazer previsão', style=ButtonStyle())

Resultado

Dia: 1
Hora: 10
Categoria: UberX
Dia: 7
Preço: R$31.68

------------------------------------------------------------

Resultado

Dia: 5
Hora: 10
Categoria: UberBlack
Dia: 15
Preço: R$53.9

------------------------------------------------------------

